# Overview of MaskMaker
### 01/16/23

1. [Introduction](#1)
2. [Overview of Documentation](#2)
3. [Structure](#3)
4. [Use](#4)
5. [Component Design](#5)

<hr>

<a id="1"></a>

## **1** Introduction

[MaskMaker](https://github.com/KollarLab/MaskMaker) contains code for drawing our coplanar-waveguide devices, and exports that drawing as a **dxf** file. Most basic components one would want to use for a device (a straight section of coplanar waveguide, a bent section of coplanar waveguide, various couplers) have been written as classes. The user can store and access information about individual components in instances of these classes, allowing for smoother and more flexible design. In addition, MaskMaker standardizes the way components are written in code, to enable more efficient collaboration in future development. All measurements are in **microns**.

<hr>

<a id="2"></a>

## **2** Documentation

Each of MaskMaker's code files contains documentation with a general description of its function that specifies, for example, its `__init__` arguments or important attributes. This documentation can also usually be accessed using `help(object)`. In addition, MaskMaker contains a few documentation files and example scripts that complement the documentation in this overview.

### **2.1** docs
#### add_component_outline
A .txt file that outlines the standardized parts of a <abbr title="MaskMaker\component">`Component`</abbr> in MaskMaker. Most new components should end up with the attributes in this outline in order to be consistent with already existing components, and to make them easy to use.
#### add_qubit_outline
A .txt file that outlines the process for adding a qubit (in their current state of development) to a design.
#### ComponentDocs
A PowerPoint presentation with diagrams of some of the more complicated components and each of their parameters.
#### example_script_outline
A .txt file that outlines the major sections a device drawing script will usually have. A “Quick Start” guide for MaskMaker using this script can be found in section **4.2**.
### **2.2** example_scripts
#### example_resonator_script
Draws a simple "shapetest" chip. A feedline is generated with integrated coupling straights, and a few lines of code at the end attach straight resonators to each hanger. The ability to reference previously placed parts like this is one of the key improvements in MaskMaker.
#### test_script_grid
Draws each MaskMaker Component with CPW straights drawn at each of its connection junctions.
#### test_script_mm_features
A script that shows off a number of MaskMaker's key capabilities, including each level of defaults, and use of junctions.
#### test_script_qubits
A script that draws qubits on CPW straights. Qubit implementation is still a work in progress.

<hr>

<a id="3"></a>

## **3** Structure
This section is an overview of the main organizational objects in MaskMaker.
### **3.1** Chip
Each MaskMaker design will begin by initializing a <abbr title="MaskMaker\mask">`Chip`</abbr>, which stores the dxf objects of the drawing, the current position and direction while adding components, and the defaults for the design. As each new <abbr title="MaskMaker\component">`Component`</abbr> is initialized, it will take a `Chip` as its `structure` argument and add polylines to `Chip.drawing`. At the end of the script, `Chip` can be saved as a dxf. A simple design might look something like this:

In [ ]:
my_chip = Chip(7000,start=junction((0,0),0)) # initializes the Chip
my_bondpad = Bondpad(my_chip) # adds a few polylines for Bondpad
CPWStraight(my_chip) # attaches a CPWStraight to that Bondpad
my_chip.saveas('<path>') # saves the drawing to a dxf file

### **3.2** junction

Most components, when initialized, will need a reference position for the drawing. MaskMaker handles this using the <abbr title="MaskMaker\junction">`junction`</abbr> object, which stores a tuple of (x,y) coordinates (`junction.coords`) and a direction in degrees, with 0 along x-axis (`junction.direction`). Initializing a junction is very straightforward:

In [ ]:
my_junc = junction((0,0),90) # tuple coordinates, scalar direction

This junction can then be passed as a reference when creating objects.This junction can then be passed as a reference when creating objects. For example, this three-way coupler will be created with one of its connections at `my_junc`:

In [ ]:
my_twc = ThreeWayCoupler(my_chip,startjunc=my_junc)

If no `startjunc` is specified, most components will reference `Chip.last`, which is also a junction. That way, `startjunc` does not have to be specified with every call.

### **3.3** Component

Most of the objects in MaskMaker will inherit the superclass <abbr title="MaskMaker\component">`Component`</abbr>, and draw their designs on a `Chip`. A <abbr title="MaskMaker\component">`Component`</abbr> often takes four key arguments. Two of them we have already discussed: `structure`, which is the `Chip` it sends its Polylines to; and `startjunc`, which is a reference `junction` for its location and orientation. The other two are `settings` and `cxns_names`.

#### **3.3.1** settings
`settings` is a dictionary of parameters for the instance of whatever component is being drawn. The keys for `settings` will vary between components. However, the possible settings for any component can be accessed using:

In [1]:
from bondpad import Bondpad
Bondpad._defaults

{'pinw': 20,
 'gapw': 8.372,
 'launcher_pinw': 400,
 'launcher_gapw': 167.44,
 'taper_length': 300,
 'launcher_padding': 167.44,
 'bond_pad_length': 350,
 'spec': 'auto'}

In addition, the a description of these settings can be accessed using:

In [ ]:
help(Bondpad)

Okay, brace yourself. MaskMaker actually has four levels of settings. In order of precedence:
1. Settings passed in a dictionary as an argument of `Component.__init__`, as discussed.
2. Chip level defaults for the component set in a dictionary at `Chip.defaults['<ComponentName>']['<default_name>']`
3. Chip level defaults for all components set in `Chip.defaults['<default_name>']`. Acceptable keys include `'pinw'`, `'gapw'`, and `'radius'` for now.
4. Component level defaults stored in `Component._defaults`.

The first level of settings we have already discussed. Because it takes precedence, any settings in `settings` will override default settings. However, if no `settings` dictionary is passed, MaskMaker will fill in the settings according to this hierarchy. The second level are default settings that apply to all instances of a specific `Component`. For example, to make every `Bondpad` drawn on your `Chip` much too long by default, use

In [ ]:
Chip.defaults['Bondpad']['bond_pad_length'] = 700

The third level of defaults handles parameters that are often required to be equal across components in order to preserve continuity or cohesiveness in the device. For example, to make every bend default to a radius of 90 microns, use

In [ ]:
Chip.defaults['radius'] = 90

The fourth level of defaults is the "last resort" for the `Component`. Each component class has a `_defaults` attribute, which always guarantee that the `__init__` call will have values for every setting.

The <abbr title="MaskMaker\component">`Component`</abbr> class is what actually handles all of these settings, and so most of the objects in MaskMaker inherit `Component`.

#### **3.3.2** cxns_names and cxns

`cxns_names` is a list of keys for `Component.cxns`, a dictionary that stores junctions at the component's connection points. These can then be referenced later using:

In [ ]:
twc_A = my_twc.cxns['connA']

The referenced junction can be used normally, as a `startjunc` or otherwise.

In [ ]:
CPWStraight(my_chip,startjunc=twc_A)

<hr>

<a id="4"></a>

## **4** Use

This section contains all of the information needed to configure MaskMaker and use it to design a chip.

### **4.1** Configuration

In order to begin using MaskMaker, GitHub must be downloaded. The repository can be cloned in Git Bash using

However, it is recommended to use GitHub Desktop, especially if you have less experience using Git. In GitHub Desktop, you can clone the repository by clicking dropdown menus for **Current repository-->Add-->Clone respository**. If you are opening GitHub Desktop for the first time, click **Clone a repository from the Internet**. There will then be an option to input an URL, which is of course **https://github.com/KollarLab/MaskMaker**.

After MaskMaker has been cloned, run <abbr title="MaskMaker\">**config.py**</abbr> in the terminal for your Python environment. This should add MaskMaker to your Python path. In order to check that the script has executed correctly, change your working directory, and then try to import one of the MaskMaker objects:

In [2]:
from bondpad import Bondpad

If it doesn't quite work, which is common on Anaconda installations, the fix is usually to run **config.py** again, and make sure that the working directory is set to **:\GitHub\MaskMaker**.

Finally, `ezdxf` will have to be installed. This can be done via:

### **4.2** General Outline for Design

This section will take you through an outline that most design scripts will follow. The code used for this section is copied from <abbr title="MaskMaker\docs\">**example_script_outline**</abbr>.

First, we import all of the classes needed. <abbr title="MaskMaker\mask">`Chip`</abbr> and `junction` will almost always be imported, and the others will be imported according to the device's need.

In [ ]:
from mask import Chip, ChipBorder
from junction import junction

from bondpad import Bondpad
from couplers.coupling_tee import CouplingStraight
from couplers.finger_cap import FingerCap
from couplers.gap_cap import GapCap
from cpw.CPWStraight import CPWStraight

Then, we will set the default settings for the design, which will eventually stored in `chip.defaults`. If we want to change `'pinw'`,`'gapw'`, or `'radius'` for *every* component in the device, we can do it here:

In [10]:
defaults = {}
defaults['pinw'] = 20
defaults['gapw'] = 8.372
defaults['radius'] = 100

Then we can set the default settings for specific components.

In [ ]:
defaults['Bondpad'] = {} # set bondpad defaults
defaults['Bondpad']['taper_length'] = 400
...
defaults['Bondpad']['bond_pad_length'] = 350

defaults['GapCap'] = {} # set gapcap defaults
...
defaults['FingerCap'] = {} # set fingercap defaults

All of the defaults are now stored in this nested dictionary `defaults`. Now we can initialize the `Chip` for our device, and set its `defaults`.

chip = Chip(7000)
chip.defaults = defaults
ChipBorder(chip) # draws a chip border

Once the `Chip` is initialized, we can start to draw components. We didn't specify `start` for our `Chip`, so it will default to `junction((0,0),0)`. In order to start somewhere else, we create a new junction `startjunc1` (see documentation for <abbr title="MaskMaker\bondpad">`Bondpad`</abbr>).

In [ ]:
startjunc1 = junction((1000,1000),0) # tuple coords, scalar direction
bondpad1 = Bondpad(chip, startjunc = startjunc1)
CPWStraight(chip,settings = {'length':200})
...
threewaycap1 = ThreeWayCoupler(chip)
...
bondpad2 = Bondpad(chip, settings = {'bond_pad_length':700})

Note that it is not necessary to specify every setting for each call, only the settings that differ from default settings. As described earlier, we can connect to stored components using the `.cxns` attribute.

In [ ]:
startjunc3 = threewaycap1.cxns['connC']
CPWStraight(chip,startjunc=startjunc3,settings = {'length':300})

Finally, we save the drawing to a dxf file.

In [ ]:
saveDir = r'Z:\Users\Theo\CAD'
filename = 'my_chip.dxf'
chip.saveas(os.path.join(saveDir,filename))

### **4.3** Qubit Pockets and Paddles

The implementation of qubits is still in development. At the moment, MaskMaker has the capability to draw trapezoidal qubit pockets and one type of digitated qubit paddles. It cannot draw any of the E-beam layers. For an outline of a script that implements qubits, see <abbr title="MaskMaker\docs\">**add_qubit_outline**</abbr>.

MaskMaker contains three potentially useful qubit classes: <abbr title="MaskMaker\qubits\notch">`QubitNotchFromJunc`</abbr> draws a simple trapezoidal pocket. <abbr title="MaskMaker\qubits\digitated_qubit">`DigitatedQubit`</abbr> draws a digitated qubit design with a cutout on one end for the qubit write, and then calls `QubitNotchFromJunc` to create a pocket around it. However, the most important qubit class is <abbr title="MaskMaker\qubits\custom_qubit">`CustomQubit`</abbr>. This allows the user to import a qubit from a dxf file and package it inside of a `QubitNotchFromJunc`. For an example of its usage, see <abbr title="MaskMaker\example_scripts\">**test_script_custom_qubits**</abbr>.

Currently, qubits are added to CPW elements by specifying a reference junction at a connection point of the CPW element. As seen in <abbr title="MaskMaker\docs\">**ComponentDocs**</abbr>, the qubit notch is drawn behind the reference junction, so that passing a connection junction of a `CPWStraight` draws the notch on that straight.

### **4.4** Importing DXF Files

MaskMaker also has the capability to import any DXF file onto a <abbr title="MaskMaker\mask">`Chip`</abbr>, using the class <attr title="MaskMaker\import_dxf">`ImportedDXF`</attr>. The imported DXF can be scaled/reflected, translated, or rotated. The call is quite straightforward, and functions like any other <abbr title="MaskMaker\component">`Component`</abbr>.

In [2]:
ImportedDXF(chip,settings={'dxf_path':my_dxf_path,'offset':(1000,1000),'xscale':10,'yscale':10,'rotation':-30})
ImportedDXF._defaults

{'xscale': 1, 'yscale': 1, 'offset': (0, 0), 'rotation': 0, 'dxf_path': None}

Examples of `ImportedDXF` are showcased in <abbr title="MaskMaker\example_scripts\">**test_script_custom_qubits**</abbr>.

### **4.5** Miscellaneous

These functions are not strictly necessary to MaskMaker, but are convenient and often helpful. They have all been grandfathered in from previous versions of MaskMaker.

#### <abbr title="MaskMaker\">alphanum</abbr>

Contains functions for drawing polyline letters. Often useful for labelling masks and chips.

#### <abbr title="MaskMaker\">pt_operations</abbr>

A number of operations on tuple (x,y) points. Often used when designing a new component using polylines, instead of existing CPW components.

<hr>

<a id="5"></a>

## **5** New Component Design

Although the previous sections should give a working understanding for using existing components, they does not encapsulate all that one needs to know to design and add components. This section will be a step-by-step walkthrough for writing a <abbr title="MaskMaker\component">`Component`</abbr>, and is essentially a better enunciated version of <abbr title="MaskMaker\docs\">**add_component_outline**</abbr>.

Each `Component` is stored within its own file, for the most part. At the top of the file there are a number of imports usually necessary:

In [ ]:
from pt_operations import rotate_pt, rotate_pts
from junction import junction
from component import Component

Then, inherit `Component` and specifiy the component name. This will also be the key used to store Chip level defaults for this new component (`Chip.defaults['<ComponentName>']['<default_name>']`). Under the class definition should be a block comment with a short description of the component and each of its settings. We do this so that `help(MyComponent)` works correctly.

In [ ]:
class ComponentName(Component):
    """
    
    << description of component and settings >>
    
    """

Under this, we need to fully specify the `_defaults` dictionary. Every setting must have a value in `_defaults` so that the component can always be drawn.

In [ ]:
    _defaults = {}
    _defaults['key1'] = val1
    _defaults['key2'] = val2
    ...

Then, we initalize the component. For most components, this will be the only method. The `__init__` call should only have five arguments: `self`, `structure`, `startjunc`, `settings` and `cxns_names`. `structure` should be required, `startjunc` should default to `None`, and `settings` should default to `{}`.

In [ ]:
    def __init__(self, structure,startjunc=None, settings = {}, cxns_names=['cxn1name','cxn2name']):
        
        s=structure

Now, we will handle the `Component.__init__()` call. This block almost always looks like:

In [ ]:
        comp_key = 'ComponentName'
        global_keys = ['pinw','gapw']
        object_keys = ['cxn_pinw','cxn_gapw']
        Component.__init__(self,s,comp_key,global_keys,object_keys,settings)
        settings = self.settings

The names `comp_key`, `global_keys`, and `object_keys` are unfortunately confusing, but it is import to understand what each of them are. `Component.__init__()` essentially needs to know three things:
1. Where to look for the `Chip` level defaults for each instance of this component. It tries the key `comp_key` in `Chip.defaults`.
2. What global defaults (`pinw`,`gapw`,`radius`) are relevant to this design.
3. What component settings are to be replaced by the global defaults.

In order to explain what I mean by those last two, it is probably easiest to just copy and paste the code from `Component` that handles this:

In [ ]:
global_defaults = {}
for n in range(len(global_keys)):
    if global_keys[n] in s.defaults: # s is a Chip
        global_defaults[object_keys[n]] = s.defaults[global_keys[n]]

`global_keys` and `object_keys` must be the same length. Our dummy component has settings `cxn_pinw` and `cxn_gapw` that will be replaced by the global `pinw` and `gapw`. I added `cxn_` in front of the setting names to emphasize that this type of default is normally used to ensure the coplanar waveguides in the device are always continuous where different components connect.

We can almost begin drawing the component. First, we have to handle `startjunc`. If the user doesn't specify one, we access `Chip.last`. Otherwise, we set `Chip.last` to the `startjunc`. It's preferable to use `.copyjunc()` when setting junctions equal to one another because it creates a new instance of <abbr title="MaskMaker\junction">`junction`</abbr>.

In [ ]:
        if startjunc is None:
            startjunc = s.last.copyjunc()
        else:
            s.last = startjunc.copyjunc()

We can draw using two methods. First, we can directly draw polylines using `Chip.drawing.add_lwpolyline(pts)`. The simple CPW components (<abbr title="MaskMaker\cpw\CPWStraight">`CPWStraight`</abbr>,<abbr title="MaskMaker\cpw\CPWLinearTaper">`CPWLinearTaper`</abbr>) use this method, and can be used as a reference.

In [ ]:
        pts = [(x1,y1),(x2,y2),(x3,y3),(x4,y4)]    
        pts=rotate_pts(pts,startjunc.direction,startjunc.coords)
        s.drawing.add_lwpolyline(pts)

Note a couple of things:
1. `pts` is a list of tuple coordinates that are connected together in the order of the list. Fairly straightforward. Don't forget to add the first point again for a closed loop.
2. Often, we can specify `pts` as if the component was anchored as (0,0) and not tilted. Then, we can make a call to <abbr title="MaskMaker\pt_operations">`rotate_pts`</abbr> or <abbr title="MaskMaker\pt_operations">`orient_pts`</abbr> in order to adjust for the actual position and orientation of the component.

We can also use existing CPW components. This can be used to make simple components, as in `TaperedCap`, but also more complex components, as in `ThreeWayCoupler`.

In [ ]:
        CPWStraight(s,settings = {'length':300,
                                  'pinw':20,
                                  'gapw':8.372
                                  })
        CPWLinearTaper(s, settings = {'length': 300,
                                      'stop_pinw': 2*20,
                                      'stop_gapw': 2*8.372
                                      })

Often this can be a bit less of a headache, because there is no need to do the extra `orient_pts` as you often must with polylines. For a good example of a component that combines both approaches to drawing, see `FingeredCap`.

Then, we set the correct position for `Chip.last`, in case it is different from where we ended up after drawing, and set the correct <abbr title="MaskMaker\junction">`junction`</abbr> for each of the connections in `.cxns`.

In [ ]:
        stopjunc = <junction>
        s.last = stopjunc

        self.cxns = {cxns_names[0]:startjunc.reverse(), cxns_names[1]:stopjunc}

With that, our `Component` is now complete! To summarize, here is a checklist of requirements for a `Component`, in order for it to be compatible with MaskMaker:
- Required arguments: `structure, startjunc=None, settings={}, cxns_names=['cxn1name','cxn2name']`
- Fully specify "last-resort" default settings in `._defaults`
- `Component.__init__()` block, with `comp_key`, `global_keys`, and `object_keys`
- `startjunc` block, which determines behavior when `startjunc` is and isn't passed
- Set `Chip.last` so that design can continue
- Set `.cxns` with keys `cxns_names` and values that are `junctions` at each of the object's connection points